In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_SILENT"] = 'True'
import warnings
warnings.filterwarnings('ignore')
import random
import matplotlib.pyplot as plt
import numpy as np
#from catalyst import utils
import cv2
import glob
from PIL import Image
import os.path as osp
from tqdm import tqdm
from typing import Callable, List, Tuple
import torch
#import catalyst
#import wandb

In [132]:
import numpy as np
from PIL import Image
img_color = Image.open('C:/Users/piai/Desktop/NationalPark/test_satellite/satellite_512_labels/사본 -AnyConv.com__LC_SN_36601_001.png')

In [118]:
len(np.array(img_color))

512

In [2]:
masking_path = 'C:/Users/piai/Downloads/airial_512_labels'
save_path = 'D:/get_dummy'

mask_file_list = os.listdir(masking_path)

for mask_file in mask_file_list:
    cur_file = os.path.join(masking_path, mask_file)
    tmp_list = []
    for i in np.array(Image.open(cur_file)).reshape(-1):
        tmp_list.append([i,i,i])
    save_file_path = os.path.join(save_path, mask_file)
    print(save_file_path)
    Image.fromarray(np.array(tmp_list).reshape(512,512,3)).save(save_file_path)

D:/get_dummy\LC_AP_36713045_047.tif
D:/get_dummy\LC_AP_36713045_048.tif
D:/get_dummy\LC_AP_36713045_049.tif
D:/get_dummy\LC_AP_36713045_050.tif
D:/get_dummy\LC_AP_36713045_051.tif
D:/get_dummy\LC_AP_36713045_052.tif
D:/get_dummy\LC_AP_36713045_053.tif
D:/get_dummy\LC_AP_36713045_054.tif
D:/get_dummy\LC_AP_36713045_055.tif
D:/get_dummy\LC_AP_36713045_056.tif
D:/get_dummy\LC_AP_36713045_057.tif
D:/get_dummy\LC_AP_36713045_058.tif
D:/get_dummy\LC_AP_36713045_059.tif
D:/get_dummy\LC_AP_36713045_060.tif
D:/get_dummy\LC_AP_36713045_061.tif
D:/get_dummy\LC_AP_36713045_062.tif
D:/get_dummy\LC_AP_36713045_063.tif
D:/get_dummy\LC_AP_36713045_064.tif
D:/get_dummy\LC_AP_36713045_065.tif
D:/get_dummy\LC_AP_36713045_066.tif
D:/get_dummy\LC_AP_36713045_067.tif
D:/get_dummy\LC_AP_36713045_068.tif
D:/get_dummy\LC_AP_36713045_069.tif
D:/get_dummy\LC_AP_36713045_070.tif
D:/get_dummy\LC_AP_36713045_071.tif
D:/get_dummy\LC_AP_36713045_072.tif
D:/get_dummy\LC_AP_36713045_073.tif
D:/get_dummy\LC_AP_36713045_

In [4]:
class UAVidColorTransformer:
    def __init__(self):
    # color table.
        self.clr_tab = self.createColorTable()
    # id table.
        id_tab = {}
        for k, v in self.clr_tab.items():
            id_tab[k] = self.clr2id(v)
        self.id_tab = id_tab

    def createColorTable(self):
        clr_tab = {}
        clr_tab['background'] = [100, 100, 100]
        clr_tab['Building'] = [10, 10, 10]
        clr_tab['Parking'] = [20, 20, 20]
        clr_tab['Road'] = [30, 30, 30]
        clr_tab['RoadSideTree'] = [40, 40, 40]
        clr_tab['LicePad'] = [50, 50, 50]
        clr_tab['FarmField'] = [60, 60, 60]
        clr_tab['Forrest'] = [70, 70, 70]
        clr_tab['NotUsing'] = [80, 80, 80]


        return clr_tab

    def colorTable(self):
        return self.clr_tab
   
    def clr2id(self, clr):
        return clr[0]+clr[1]*255+clr[2]*255*255

#transform to uint8 integer label
    def transform(self,label, dtype=np.int32):
        height,width = label.shape[:2]
    # default value is index of clutter.
        newLabel = np.zeros((height, width), dtype=dtype)
        id_label = label.astype(np.int64)
        id_label = id_label[:,:,0]+id_label[:,:,1]*255+id_label[:,:,2]*255*255
        for tid,val in enumerate(self.id_tab.values()):
            mask = (id_label == val)
            newLabel[mask] = tid
        return newLabel

  #transform back to 3 channels uint8 label
    def inverse_transform(self, label):
        label_img = np.zeros(shape=(label.shape[0], label.shape[1],3),dtype=np.uint8)
        values = list(self.clr_tab.values())
        for tid,val in enumerate(values):
            mask = (label==tid)
            label_img[mask] = val
        return label_img

In [5]:
clrEnc = UAVidColorTransformer()
def prepareTrainIDForDir(gtDirPath, saveDirPath):
    gt_paths = [p for p in os.listdir(gtDirPath) if p.startswith('seq')]
    for pd in tqdm(gt_paths):
        lbl_dir = osp.join(gtDirPath, pd, 'Labels')
        lbl_paths = os.listdir(lbl_dir)
        if not osp.isdir(osp.join(saveDirPath, pd, 'TrainId')):
            os.makedirs(osp.join(saveDirPath, pd, 'TrainId'))
            assert osp.isdir(osp.join(saveDirPath, pd, 'TrainId')), 'Fail to create directory:%s'%(osp.join(saveDirPath, pd, 'TrainId'))
        for lbl_p in lbl_paths:
            print(lbl_p)
            lbl_path = osp.abspath(osp.join(lbl_dir, lbl_p))
            trainId_path = osp.join(saveDirPath, pd, 'TrainId', lbl_p)
            gt = np.array(Image.open(lbl_path))
            trainId = clrEnc.transform(gt, dtype=np.uint8)
            Image.fromarray(trainId).save(trainId_path)

In [6]:
masking_path = 'D:/get_dummy'
save_path = 'C:/Users/piai/Downloads/airial_512_labels'

mask_file_list = os.listdir(masking_path)

for mask_file in mask_file_list:
    cur_file = os.path.join(masking_path, mask_file)
    gt = np.array(Image.open(cur_file))
    trainId = clrEnc.transform(gt, dtype=np.uint8)

    save_file_path = os.path.join(save_path, mask_file)
    print(save_file_path)
    print(np.unique(trainId))
    print('\n')
    Image.fromarray(trainId).save(save_file_path)

C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_047.tif
[0 1 3 5 6 7]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_048.tif
[0 1 3 5 6]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_049.tif
[0 5 6]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_050.tif
[0 1 5 6]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_051.tif
[0 1 5 6 7]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_052.tif
[0 1 5 6 7]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_053.tif
[0 1 5 6 7]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_054.tif
[0 1 5 6 7]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_055.tif
[0 5 6 7]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_056.tif
[0 1 2 3 5 6 7 8]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_057.tif
[0 1 2 6 7]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36713045_058.tif
[0 1 5 6 7 8]


C:/Users/piai/Downloads/airial_512_labels\LC_AP_36